# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup

#visualization
import matplotlib.pyplot as plt
import seaborn as sns

#data science - standard liberary
import pandas as pd
import numpy as np

#requests
import requests
import time
from pprint import pprint

#regression
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import scipy.stats as sts

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

604

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
cities_sub=cities[0:10]
cities_sub                

['saskylakh',
 'te anau',
 'busselton',
 'puerto ayora',
 'barentsburg',
 'hirado',
 'atuona',
 'bluff',
 'jamestown',
 'rikitea']

In [4]:

#init lists to hold parsed data
lats = []
longs = []
temps = []
feel_likes = []
humids = []
pressures = []
speeds = []
cities_found = []
cloudiness = []

for i in range(len(cities)):
    #get the city
    city = cities[i]
    
    #create the url
    units = "imperial"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units={units}&appid={weather_api_key}"
#     print(url)

    #make the request
    response = requests.get(url)
#     print(response.status_code)
    
     #error check
    if response.status_code == 200:
        data = response.json()
#         pprint(data)
        
        try:
            #extract the data
            lat = data["coord"]["lat"]
            long = data["coord"]["lon"]
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            pressure = data["main"]["pressure"]
            speed = data["wind"]["speed"]
            clouds = data["clouds"]["all"]
            city_name = data["name"]

            #save the data
            lats.append(lat)
            longs.append(long)
            temps.append(temp)
            feel_likes.append(feels_like)
            humids.append(humidity)
            pressures.append(pressure)
            speeds.append(speed)
            cloudiness.append(clouds)
            cities_found.append(city_name)
        except Exception as e:
            print(f"Through exception for city {city}: {e}")
    
    elif response.status_code == 404:
        print(f"Missing data in OpenWeatherAPI for {city}")
    else:
        print(response.status_code)
        print("Oh gosh darnit. The API is broken. Sad Face.")
        
        
        
    #print every 5
    if (i % 5 == 0):
        print(f"Got data for city index: {i} of {len(cities)}")
    
    
    #sleep for requests
    time.sleep(1)


Got data for city index: 0 of 604
Missing data in OpenWeatherAPI for barentsburg
Got data for city index: 5 of 604
Got data for city index: 10 of 604
Missing data in OpenWeatherAPI for khokholskiy
Got data for city index: 15 of 604
Missing data in OpenWeatherAPI for illoqqortoormiut
Got data for city index: 20 of 604
Missing data in OpenWeatherAPI for taolanaro
Got data for city index: 25 of 604
Got data for city index: 30 of 604
Got data for city index: 35 of 604
Missing data in OpenWeatherAPI for nizhneyansk
Missing data in OpenWeatherAPI for mouzakion
Got data for city index: 40 of 604
Missing data in OpenWeatherAPI for ijaki
Got data for city index: 45 of 604
Got data for city index: 50 of 604
Got data for city index: 55 of 604
Got data for city index: 60 of 604
Missing data in OpenWeatherAPI for attawapiskat
Got data for city index: 65 of 604
Got data for city index: 70 of 604
Got data for city index: 75 of 604
Got data for city index: 80 of 604
Missing data in OpenWeatherAPI for 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
# make the dataframe
df = pd.DataFrame()
df["City"] = cities_found
df["Latitude"] = lats
df["Longitude"] = longs
df["Temperature"] = temps
df["Feels Like"] = feel_likes
df["Humidity"] = humids
df["Pressure"] = pressures
df["Wind Speed"] = speeds
df["Cloudiness"] = cloudiness

df.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Saskylakh,71.92,114.08,-7.91,-22.68,86,1008,14.38,13
1,Te Anau,-45.42,167.72,57.13,52.84,55,1018,4.05,92
2,Busselton,-33.65,115.33,60.37,53.62,82,996,14.65,97
3,Puerto Ayora,-0.74,-90.35,69.01,70.34,83,1012,5.99,11
4,Hirado,33.36,129.55,56.39,45.70,47,1022,13.87,20


In [6]:
#save as CVS fiel for later use
df.to_csv(output_data_file,index=False)
